In [3]:


# Import packages
import osgeo
from os.path import join as pjoin
import pandas as pd
import numpy as np
import xarray as xr
import xrspatial as xrs
import rioxarray

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

import pystac
from pystac_client import Client
import pystac.item_collection as pyic
import requests
import stackstac

import rich.table
import dask.diagnostics

scratch_dir = './scratch_dir' 

from pystac_client import Client
import planetary_computer as pc

In [4]:
# Define your area of interest
# Khangchendzonga National Park
aoi_KN = {
  "type": "Polygon",
  "coordinates": [
    [
      [87.65923734067707, 27.393428564097675],
      [89.2545630402231, 27.393428564097675],
      [89.2545630402231, 28.157669203196022],
      [87.65923734067707, 28.157669203196022],
      [87.65923734067707, 27.393428564097675]
    ]
  ]
}

In [2]:
# # Define your area of interest
# aoi_EMV = { #aoi Everest and Makalu Valley Polygon
#   "type": "Polygon",
#   "coordinates": [
#     [
#       [86.63380152490207, 28.56507505979343],
#       [87.74001280776696, 28.479331576213163],
#       [87.7789385926358, 28.213296130511523],
#       [87.61479871540345, 27.906713498710847],
#       [87.61479871540345, 27.906713498710847],
#       [87.04816717665705, 27.239072299890207],
#       [87.04816717665705, 27.239072299890207],
#       [28.54835734379745, 86.06605505495027],
#       [86.63380152490207, 28.56507505979343]
#     ]
#   ]
# }

In [3]:
# aoi_KV = { # aoi Kachenjunga Valley
# "type": "Polygon",
#   "coordinates": [
#     [
#     [86.63380152490207, 28.56507505979343],
#     [87.74001280776696, 28.479331576213163],
#     [87.7789385926358, 28.213296130511523],
#     [87.61479871540345, 27.906713498710847],
#     [87.61479871540345, 27.906713498710847],
#     [87.15652306810863, 27.36778735019498],
#     [89.06841422478996, 27.269554114998044],
#     [89.06841422478996, 28.56507505979343],
#     [86.63380152490207, 28.56507505979343]
#     ]
#   ]
# }

In [19]:
daterange = [
    # {"interval": ["2000-08-01T00:00:00Z", "2000-11-30T23:59:59Z"]},
    # {"interval": ["2005-08-01T00:00:00Z", "2005-11-30T23:59:59Z"]},
    # {"interval": ["2010-08-01T00:00:00Z", "2010-11-30T23:59:59Z"]},
    {"interval": ["2015-08-01T00:00:00Z", "2015-11-30T23:59:59Z"]},
    {"interval": ["2020-08-01T00:00:00Z", "2020-11-30T23:59:59Z"]}]

def importdata(aoi, daterange):
    catalog = Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
        )
    # Define your search with CQL2 syntax
    search = catalog.search(filter_lang="cql2-json", filter={
        "op": "and",
        "args": [{"op": "s_intersects", "args": [{"property": "geometry"}, aoi_KN]},
                {"op": "anyinteracts", "args": [{"property": "datetime"}, daterange[0]]},
                {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
                {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 2]}
                ]
        }
    )


    first_item = next(search.get_items())
    pc.sign_item(first_item).assets

    charts = search.get_all_items()
    print(daterange[0],len(charts))
               
        #########################
    for t in daterange[1:]:
         # Search against the Planetary Computer STAC API
        catalog = Client.open(
            "https://planetarycomputer.microsoft.com/api/stac/v1")
        # Define your search with CQL2 syntax
        search = catalog.search(filter_lang="cql2-json", filter={
            "op": "and",
                "args": [
                    {"op": "s_intersects", "args": [{"property": "geometry"}, aoi_KN]},
                    {"op": "anyinteracts", "args": [{"property": "datetime"}, t]},
                    {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
                    # data cant process more than k-amount of bands so cloud coverage can be set to almost 0
                    {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 2]} 
                ]
            }  
        )

        first_item = next(search.get_items())
        pc.sign_item(first_item).assets

        items = search.get_all_items()
        print(t,'items: ',len(items))
        charts = charts+items

    print('Length total item set:',len(charts))
    return charts     

charts = importdata(aoi_KN, daterange)

{'interval': ['2015-08-01T00:00:00Z', '2015-11-30T23:59:59Z']} 11
{'interval': ['2020-08-01T00:00:00Z', '2020-11-30T23:59:59Z']} items:  11
Length total item set: 22


In [20]:
ds = stackstac.stack(pc.sign(charts), epsg=6207)
ds

<xarray.DataArray 'stackstac-76b57e66548be660d3b748b0d049686f' (time: 22,
                                                                band: 22,
                                                                y: 7813,
                                                                x: 13449)>
dask.array<fetch_raster_window, shape=(22, 22, 7813, 13449), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/31)
  * time                         (time) datetime64[ns] 2015-09-07T04:35:26.64...
    id                           (time) <U31 'LC08_L2SP_139040_20150907_02_T1...
  * band                         (band) <U13 'qa' 'red' ... 'atmos_opacity'
  * x                            (x) float64 85.83 85.83 85.83 ... 90.02 90.02
  * y                            (y) float64 29.92 29.92 29.92 ... 27.8 27.8
    view:off_nadir               int32 0
    ...                           ...
    raster:bands                 (band) object {'unit': 'kelvin', 'scale': 0....
    classification:bitfields     (band) object None None ... None
    common_name                  (band) object None None None ... None None
    center_wavelength            (band) object None None None ... None None
    full_width_half_max          (band) object None None None ... 2.05 None None
    epsg                         int32 6207
Attributes:
    spec:           RasterSpec(epsg=6207, bounds=(85.82632443669922, 27.79862...
    crs:            epsg:6207
    transform:      | 0.00, 0.00, 85.83|\n| 0.00,-0.00, 29.92|\n| 0.00, 0.00,...
    resolution_xy:  (0.0003116537435516875, 0.00027169917465386106)

In [21]:
import rich.table

def display_asset_table(item):
    table = rich.table.Table("Asset Key", "Description")
    for asset_key, asset in item.assets.items():
        table.add_row(asset_key, asset.title)
    return table
    

# Assuming you have a stack of items called "item_stack"
first_item = charts[0]
display_asset_table(first_item)






# import rich.table

# table = rich.table.Table("Asset Key", "Description")
# for asset_key, asset in first_item.assets.items():
#     table.add_row(asset_key, asset.title)

# table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Description                                                      ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ qa               │ Surface Temperature Quality Assessment Band                      │
│ ang              │ Angle Coefficients File                                          │
│ red              │ Red Band                                                         │
│ blue             │ Blue Band                                                        │
│ drad             │ Downwelled Radiance Band                                         │
│ emis             │ Emissivity Band                                                  │
│ emsd             │ Emissivity Standard Deviation Band                               │
│ lwir             │ Surface Temperature Band                                         │
│ trad             │ Thermal Radiance Band                                            │
│ urad             │ Upwelled Radiance Band                                           │
│ atran            │ Atmospheric Transmittance Band                                   │
│ cdist            │ Cloud Distance Band                                              │
│ green            │ Green Band                                                       │
│ nir08            │ Near Infrared Band 0.8                                           │
│ swir16           │ Short-wave Infrared Band 1.6                                     │
│ swir22           │ Short-wave Infrared Band 2.2                                     │
│ mtl.txt          │ Product Metadata File (txt)                                      │
│ mtl.xml          │ Product Metadata File (xml)                                      │
│ cloud_qa         │ Cloud Quality Assessment Band                                    │
│ mtl.json         │ Product Metadata File (json)                                     │
│ qa_pixel         │ Pixel Quality Assessment Band                                    │
│ qa_radsat        │ Radiometric Saturation and Dropped Pixel Quality Assessment Band │
│ atmos_opacity    │ Atmospheric Opacity Band                                         │
│ tilejson         │ TileJSON with default rendering                                  │
│ rendered_preview │ Rendered preview                                                 │
└──────────────────┴──────────────────────────────────────────────────────────────────┘

In [22]:
from IPython.display import Image

Image(url=first_item.assets["rendered_preview"].href, width=500)

In [27]:
# Loop over file names and save each file
with dask.diagnostics.ProgressBar():
    ts = ds.compute()
    output_file = pjoin(scratch_dir, 'data.tif')
    ts.rio.to_raster(output_file, compress='LZW')

[                                        ] | 0% Completed | 33.81 sms


RuntimeError: Error reading Window(col_off=6144, row_off=1024, width=1024, height=1024) from 'https://landsateuwest.blob.core.windows.net/landsat-c2/level-2/standard/oli-tirs/2015/139/040/LC08_L2SP_139040_20150907_20200908_02_T1/LC08_L2SP_139040_20150907_20200908_02_T1_ST_EMIS.TIF?st=2023-03-29T12%3A42%3A21Z&se=2023-03-30T13%3A27%3A21Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-03-30T11%3A36%3A07Z&ske=2023-04-06T11%3A36%3A07Z&sks=b&skv=2021-06-08&sig=4L4SSCJyYUq00gOe7LpExutcMN1P/2YQl2NPjxk6hyw%3D': RasterioIOError('Read or write failed. IReadBlock failed at X offset 12, Y offset 8: IReadBlock failed at X offset 1, Y offset 4: TIFFReadEncodedTile() failed.')

In [24]:
red = ds.sel(band='red')
nir = ds.sel(band='nir08')
ndvi = (nir-red) / (nir+red)
yearly = ndvi.resample(time="AS").max("time", keep_attrs=True)
yearly

c:\Users\k\anaconda3\envs\ssml\lib\site-packages\dask\array\core.py:4828: PerformanceWarning: Increasing number of chunks by factor of 14
  result = blockwise(


<xarray.DataArray 'stackstac-76b57e66548be660d3b748b0d049686f' (time: 6,
                                                                y: 7813,
                                                                x: 13449)>
dask.array<where, shape=(6, 7813, 13449), dtype=float64, chunksize=(5, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/18)
  * x                            (x) float64 85.83 85.83 85.83 ... 90.02 90.02
  * y                            (y) float64 29.92 29.92 29.92 ... 27.8 27.8
  * time                         (time) datetime64[ns] 2015-01-01 ... 2020-01-01
    view:off_nadir               int32 0
    landsat:collection_number    <U2 '02'
    landsat:wrs_type             <U1 '2'
    ...                           ...
    raster:bands                 object {'scale': 2.75e-05, 'nodata': 0, 'off...
    classification:bitfields     object None
    common_name                  object None
    center_wavelength            object None
    full_width_half_max          object None
    epsg                         int32 6207

In [25]:
ls_vars = ['view:off_nadir', 'landsat:wrs_type', 'gsd',  
           'landsat:correction', 'classification:bitfields',
           'raster:bands', 'epsg', 'landsat:collection_number']
ts = yearly.copy()
ts = ts.drop_vars(ls_vars)

In [26]:
tss = ts.copy()
tss = tss.rename(new_name_or_name_dict = 'ndvi')
tss.attrs['long_name'] = tuple(tss.time.data.astype(str))

tss.rio.write_crs("epsg:32631", inplace=True)
tss.rio.set_spatial_dims(x_dim='x', y_dim='y', inplace=True)
tss.rio.to_raster(pjoin(scratch_dir, 'tss_ndvi.tif'), compress='LZW')

c:\Users\k\anaconda3\envs\ssml\lib\site-packages\dask\array\reductions.py:640: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
